In [2]:
import warnings
import sys
sys.path.append('../')
from Notebooks.helper import load_env

warnings.filterwarnings('ignore')
load_env()

Import all the required dependencies

In [4]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
import os
import pandas as pd

In [5]:
prompt = """
Answer the question based on the provided context.
Context: {{ context }}
Question: {{ query }}
"""
prompt_builder = PromptBuilder(template=prompt)
generator = OpenAIGenerator()

query_pipeline = Pipeline()
query_pipeline.add_component("prompt", prompt_builder)
query_pipeline.add_component("generator", generator)

query_pipeline.connect("prompt", "generator")

🚅 Components
  - prompt: PromptBuilder
  - generator: OpenAIGenerator
🛤️ Connections
  - prompt.prompt -> generator.prompt (str)

In [14]:
df = pd.read_csv('gics-map-2018.csv')
for index, row in df.iterrows():
    # Access the values of each column in the current row
    sub_industry_description = row['SubIndustryDescription']
    question = f"Give me an unformatted list of Climate related risks for a company who's industry is classified as {sub_industry_description}? Provide response in this exact JSON format: ['']"
    result = query_pipeline.run(
        {
            "prompt": {"query": question},
        }
    )
    print('Risks - '+result['generator']['replies'][0])
    df.at[row.name,'Climate Risk'] = result['generator']['replies'][0]

    question = f"Give me an unformatted list of Risk mitigations for the risks previously provided? Provide response in this exact JSON format: ['']"
    result = query_pipeline.run(
        {
            "prompt": {"query": question},
        }
    )
    print('Mitigations - '+result['generator']['replies'][0])

    df.at[row.name,'Climate Risk Mitigation'] = result['generator']['replies'][0]
    df.to_csv('Gics_modified.csv')

Risks - ['1. Impact of extreme weather events on drilling operations', '2. Risk of regulatory changes affecting drilling permits', '3. Water scarcity leading to drilling challenges', '4. Potential for reputational damage due to environmental concerns', '5. Vulnerability to shifts in demand for fossil fuels']
Mitigations - ['1. Implement regular security updates and patches', '2. Conduct regular security assessments and penetration testing']
Risks - ['1. Extreme weather events (e.g. hurricanes, storms) impacting operations', '2. Regulatory changes related to reducing carbon emissions']
Mitigations - ['response1', 'response2']
Risks - ['1. Impact of climate change on oil and gas reserves',
'2. Regulatory risks related to carbon emissions and climate change policies',
'3. Physical risks from extreme weather events impacting operations and infrastructure',
'4. Market risks due to changing consumer preferences for renewable energy sources',
'5. Litigation risks related to climate change lia

KeyboardInterrupt: 

Set up the document ingestion pipeline

In [ ]:
print(result["generator"]["replies"][0])

{
    "industries": ["Bicycle industry", "Retail industry"]
}


In [ ]:
industry_analysis_pipeline = Pipeline()